# Customer 90-Day Purchase Propensity & Revenue Prediction

##  Problem Statement
Retail businesses lose revenue when they cannot identify which customers are likely to return.
This project aims to predict:
1.  **Propensity:** Will the customer purchase in the next 90 days? (Classification)
2.  **Revenue:** How much will they spend? (Regression)

##  Objectives
* **Segment Customers** based on Recency, Frequency, and Monetary (RFM) scores.
* **Engineer Features** using a temporal split (past behavior predicts future behavior).
* **Build & Evaluate Models** using PyCaret to maximize ROI.

In [ ]:
import pandas as pd
import pycaret.classification as clf
import pycaret.regression as reg

# Constants
profit_margin = 0.15  # 15% Profit on Products

##  Dataset Description
The dataset contains transactional-level data where each row is a single purchase event.
Key columns:
- **UserId**: Unique identifier for the customer.
- **TransactionTime**: Timestamp of the purchase.
- **NumberOfItemsPurchased**: Quantity of items in the transaction.
- **CostPerItem**: Price per individual item.

In [ ]:
# Load data
df = pd.read_csv('../data/transaction_data.csv')

# Ensure the 'TransactionTime' column is in datetime format
df['TransactionTime'] = pd.to_datetime(df['TransactionTime'])
df['sales_value'] = df['NumberOfItemsPurchased'] * df['CostPerItem']

print(f"Data Loaded: {df.shape[0]} rows, {df['UserId'].nunique()} unique users.")

##  Temporal Train-Test Split
To avoid **data leakage**, we cannot use a standard random split. We must use a time-based split.
* **Training Data:** All transactions *before* the cutoff date.
* **Target Data:** Did the user purchase *after* the cutoff date?
This ensures we are simulating a real-world scenario where we predict future behavior based on past history.

In [ ]:
# Set the prediction window for the next 90 days
prediction_window_days = 90
latest_date = df['TransactionTime'].max()
prediction_cutoff = latest_date - pd.to_timedelta(prediction_window_days, unit="d")

# Train-Test Split
temporal_in_df = df[df['TransactionTime'] < prediction_cutoff]
temporal_out_df = df[(df['TransactionTime'] > prediction_cutoff) & 
                      (df['UserId'].isin(temporal_in_df['UserId']))]

print(f"Prediction Cutoff Date: {prediction_cutoff}")

##  Feature Engineering
We transform raw transactions into behavioral features:
* **Recency:** Days since last purchase (Active vs. Dormant).
* **Frequency:** Total count of past purchases (Loyalty).
* **Monetary:** Total and Average spend (High-value customers).
* **Short-term Activity:** Transactions in the last 14 and 28 days (Recent engagement signals).

In [ ]:
# Target Creation - For the next 90 days
targets_df = (
    temporal_out_df.groupby("UserId")["sales_value"]  # Select only the numeric column
    .sum()
    .rename("sales_90_value")
    .reset_index()  # Reset index to flatten the DataFrame
)

# Add a flag column to indicate these users made purchases in the 90-day period
targets_df["sales_90_flag"] = 1

# Recency Feature Creation - Time since last purchase date for each customer
max_date = temporal_in_df["TransactionTime"].max()
recency_features_df = temporal_in_df[["UserId", "TransactionTime"]] \
    .groupby("UserId") \
    .apply(lambda x: int((max_date - x["TransactionTime"].max()) / pd.to_timedelta(1, "day"))) \
    .to_frame(name="recency")

# Frequency Feature Creation - Count of purchases for each customer
frequency_features_df = temporal_in_df[["UserId", "TransactionTime"]] \
    .groupby("UserId") \
    .count() \
    .rename(columns={"TransactionTime": "frequency"})

# Monetary Feature Creation - Sum and Mean of sales for each customer
monetary_features_df = temporal_in_df[["UserId", "sales_value"]] \
    .groupby("UserId") \
    .agg(sales_value_sum=("sales_value", "sum"), 
         sales_value_mean=("sales_value", "mean"))

# Transactions in the Last Month (28 days)
cutoff_28d = prediction_cutoff - pd.to_timedelta(28, unit="d")
transactions_last_month_df = temporal_in_df[['UserId', 'TransactionTime']] \
    .drop_duplicates() \
    .query("TransactionTime > @cutoff_28d") \
    .groupby("UserId") \
    .size() \
    .to_frame(name='transactions_last_month')

# Transactions in the Last 2 Weeks (14 days)
cutoff_14d = prediction_cutoff - pd.to_timedelta(14, unit="d")
transactions_last_2weeks_df = temporal_in_df[["UserId", "TransactionTime"]] \
    .drop_duplicates() \
    .query("TransactionTime > @cutoff_14d") \
    .groupby("UserId") \
    .size() \
    .to_frame(name='transactions_last_2weeks')

# Spend in the Last 2 Weeks
sales_last_2weeks_df = (
    temporal_in_df[["UserId", "TransactionTime", "sales_value"]]
    .drop_duplicates()
    .query("TransactionTime > @cutoff_14d")
    .groupby("UserId", as_index=False)["sales_value"]  # Restrict to numeric column
    .sum()
    .rename(columns={"sales_value": "sales_value_last_2weeks"})
)

# Combine all features into a single DataFrame
features_df = recency_features_df \
    .merge(frequency_features_df, on="UserId", how="left") \
    .merge(monetary_features_df, on="UserId", how="left") \
    .merge(transactions_last_month_df, on="UserId", how="left") \
    .merge(transactions_last_2weeks_df, on="UserId", how="left") \
    .merge(sales_last_2weeks_df, on="UserId", how="left") \
    .merge(targets_df, on="UserId", how="left").fillna(0)

print("Features Created Successfully. Sample:")
features_df.head()

##  Predictive Modeling

### Model 1: Purchase Propensity (Classification)
**Goal:** Predict *if* a customer will buy in the next 90 days (Yes/No).
**Metric:** AUC and Precision (to minimize false positives in marketing campaigns).

In [ ]:
# Initialize the PyCaret classification setup
clf_setup = clf.setup(
    data=features_df,
    target='sales_90_flag',
    session_id=123,
    verbose=False
)

# Compare and select the best model
best_clf_model = clf.compare_models()

# Train the best model
final_clf_model = clf.finalize_model(best_clf_model)

print("Best Classification Model:", final_clf_model)

### Model 2: Revenue Prediction (Regression)
**Goal:** Estimate *how much* a customer will spend.
**Metric:** RMSE (Root Mean Square Error).

In [ ]:
# For regression modeling on 'sales_90_value' (predicted sales amount in 90 days)
reg_setup = reg.setup(
    data=features_df,
    target='sales_90_value',
    session_id=123,
    verbose=False
)

# Compare and select the best regression model
best_reg_model = reg.compare_models()

# Train the best regression model
final_reg_model = reg.finalize_model(best_reg_model)

print("Best Regression Model:", final_reg_model)

##  Business Insights & Conclusion
1.  **Recency is Key:** Customers who bought recently are exponentially more likely to buy again.
2.  **Frequency matters for Stability:** High-frequency buyers have lower variance in their predicted revenue.
3.  **Actionable Strategy:**
    * **High Propensity / High Revenue:** VIP treatment (Exclusive offers).
    * **High Propensity / Low Revenue:** Upsell campaigns (Bundles).
    * **Low Propensity:** Win-back campaigns (if Recency < 90 days) or ignore (if churned).